# Task 2.2: Getting Started with tweepy
## Inicialització
En primer lloc importem tweepy, llibreria de Python que ens permetra accedir a la API de Twitter, en aquesta sessió API REST.

In [2]:
import tweepy
from tweepy import OAuthHandler

consumer_key = "cNbkKHve3645xLIVgJjQbrWg6"
consumer_secret = "B1SWKCIMFQjhh3Y5YGstKMvl8t7IAPTha1jxXy5b3IC6K8wz7w"
access_token = "3720533727-cBbDc2bDnDSU6f9SuC3ti4oQcfTkMk7VbVmh9ZW"
access_secret = "k2aioMNx3tCfw6jW9teksQ98WllWbKogVQF4YW0sRYTuW"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

## Printat de propietats
Imprimim les dades relacionades amb el nostre compte (es reutilitza un compte de TFG).

In [3]:
user = api.me()
 
print('Name: ' + user.name)
print('Location: ' + user.location)
print('Friends: ' + str(user.followers_count))
print('Created: ' + str(user.created_at))
print('Description: ' + str(user.description))

Name: twiterbot_noticies
Location: 
Friends: 2
Created: 2015-09-20 19:16:41
Description: 


## Printat d'un tweet
Procedim a imprimir l'últim tweet creat per un amic (paràmetre 1 indica el número dels últims tweets a obtenir).

In [4]:
for status in tweepy.Cursor(api.home_timeline).items(1):
    print(status.text) 

L'estat ha d'acollir 17.337 persones abans del 26/9. #VeniuJa ho recorda amb un comptador de temps.… https://t.co/2W6I4OHrsG


Fem el mateix imprimint un tweet però en aquest cas importem la llibreria de JSON i l'imprimim com a tal amb una identació de 2 espais.

In [5]:
import json

for status in tweepy.Cursor(api.home_timeline).items(1):
    print(json.dumps(status._json, indent=2))

{
  "contributors": null, 
  "truncated": true, 
  "text": "L'estat ha d'acollir 17.337 persones abans del 26/9. #VeniuJa ho recorda amb un comptador de temps.\u2026 https://t.co/2W6I4OHrsG", 
  "is_quote_status": false, 
  "in_reply_to_status_id": null, 
  "id": 863088937098768384, 
  "favorite_count": 1, 
  "source": "<a href=\"http://www.hootsuite.com\" rel=\"nofollow\">Hootsuite</a>", 
  "retweeted": false, 
  "coordinates": null, 
  "entities": {
    "symbols": [], 
    "user_mentions": [], 
    "hashtags": [
      {
        "indices": [
          53, 
          61
        ], 
        "text": "VeniuJa"
      }
    ], 
    "urls": [
      {
        "url": "https://t.co/2W6I4OHrsG", 
        "indices": [
          101, 
          124
        ], 
        "expanded_url": "https://twitter.com/i/web/status/863088937098768384", 
        "display_url": "twitter.com/i/web/status/8\u2026"
      }
    ]
  }, 
  "in_reply_to_screen_name": null, 
  "in_reply_to_user_id": null, 
  "retweet_coun

## Printat d'informació d'amistats
En aquest cas agafem un amic i printem la seva infromació en JSON, aprofitaant l'import que hem fet anteriorment.

In [6]:
for friend in tweepy.Cursor(api.friends).items(1):
    print(json.dumps(friend._json, indent=2))

{
  "follow_request_sent": false, 
  "has_extended_profile": false, 
  "profile_use_background_image": true, 
  "live_following": false, 
  "default_profile_image": false, 
  "id": 95695959, 
  "profile_background_image_url_https": "https://pbs.twimg.com/profile_background_images/653258374046478336/dw7DPLlC.jpg", 
  "translator_type": "regular", 
  "verified": true, 
  "blocked_by": false, 
  "profile_text_color": "333333", 
  "muting": false, 
  "profile_image_url_https": "https://pbs.twimg.com/profile_images/857140669839798273/QhREDVHI_normal.jpg", 
  "profile_sidebar_fill_color": "FFFFFF", 
  "entities": {
    "url": {
      "urls": [
        {
          "url": "https://t.co/FWP1q1SrbD", 
          "indices": [
            0, 
            23
          ], 
          "expanded_url": "http://www.barcelona.cat", 
          "display_url": "barcelona.cat"
        }
      ]
    }, 
    "description": {
      "urls": []
    }
  }, 
  "followers_count": 328580, 
  "profile_sidebar_border_col

# Task 2.3: Tweet pre-processing
## Tractament de paraules
En primer lloc importem el nltk.tokenize (mòdul word_tokenize) que ens permetrà el tractament de paraules.

In [7]:
from nltk.tokenize import word_tokenize

tweet = 'RT @JordiTorresBCN: just an example! :D http://JordiTorres.Barcelona #masterMEI'

print(word_tokenize(tweet))

['RT', '@', 'JordiTorresBCN', ':', 'just', 'an', 'example', '!', ':', 'D', 'http', ':', '//JordiTorres.Barcelona', '#', 'masterMEI']


## Tractament d'emoticones
Amb el següent codi tractem els emoticones que hi poden haver als tweets.

In [8]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @JordiTorresBCN: just an example! :D http://JordiTorres.Barcelona #masterMEI'
print(preprocess(tweet))

['RT', '@JordiTorresBCN', ':', 'just', 'an', 'example', '!', ':D', 'http://JordiTorres.Barcelona', '#masterMEI']


## Resultat
Com es pot comprovar en la sortida anterior, ara l'emoticona ":D" ens el tracta com una paraula.